# Private RAG Application Implementation Using Llama 3.2, Ollama & PostgreSQL

In [1]:
import psycopg2

## Dataset Preparation

In [2]:
# the knowledge base
dummy_data = [
    {"title": "Seoul Tower", "content": "Seoul Tower is a communication and observation tower located on Namsan Mountain in central Seoul, South Korea."},
    {"title": "Gwanghwamun Gate", "content": "Gwanghwamun is the main and largest gate of Gyeongbokgung Palace, in Jongno-gu, Seoul, South Korea."},
    {"title": "Bukchon Hanok Village", "content": "Bukchon Hanok Village is a Korean traditional village in Seoul with a long history."},
    {"title": "Myeong-dong Shopping Street", "content": "Myeong-dong is one of the primary shopping districts in Seoul, South Korea."},
    {"title": "Dongdaemun Design Plaza", "content": "The Dongdaemun Design Plaza is a major urban development landmark in Seoul, South Korea."}
]

## Connect to the database

In [3]:
def connect_db():
    return psycopg2.connect( # use the credentials of your postgresql database 
        host = 'localhost',
        database = 'postgres',
        user = 'postgres',
        password = 'password',
        port = '5432'
    )

## Create the table

In [4]:
conn = connect_db()
cur = conn.cursor()
cur.execute("""
        CREATE TABLE IF NOT EXISTS documents (
            id SERIAL PRIMARY KEY,
            title TEXT,
            content TEXT,
            embedding VECTOR(768)
        );
    """)
conn.commit()
cur.close()
conn.close()

### Insert the data and embeddings into the database

In [5]:
conn = connect_db()
cur = conn.cursor()

# use the port at which your ollama service is running.
for doc in dummy_data:
    cur.execute("""
        INSERT INTO documents (title, content, embedding)
        VALUES (
            %(title)s,
            %(content)s,
            ollama_embed('nomic-embed-text', concat(%(title)s, ' - ', %(content)s), _host=>'http://172.17.0.2:11434')
        )
    """, doc)

conn.commit()
cur.close()
conn.close()

Safechecking the insert information in the database.

In [6]:
conn = connect_db()
cur = conn.cursor()
    
cur.execute("""
    SELECT title, content, vector_dims(embedding) 
    FROM documents;
""")

rows = cur.fetchall()
for row in rows:
    print(f"Title: {row[0]}, Content: {row[1]}, Embedding Dimensions: {row[2]}")

cur.close()
conn.close()

Title: Seoul Tower, Content: Seoul Tower is a communication and observation tower located on Namsan Mountain in central Seoul, South Korea., Embedding Dimensions: 768
Title: Gwanghwamun Gate, Content: Gwanghwamun is the main and largest gate of Gyeongbokgung Palace, in Jongno-gu, Seoul, South Korea., Embedding Dimensions: 768
Title: Bukchon Hanok Village, Content: Bukchon Hanok Village is a Korean traditional village in Seoul with a long history., Embedding Dimensions: 768
Title: Myeong-dong Shopping Street, Content: Myeong-dong is one of the primary shopping districts in Seoul, South Korea., Embedding Dimensions: 768
Title: Dongdaemun Design Plaza, Content: The Dongdaemun Design Plaza is a major urban development landmark in Seoul, South Korea., Embedding Dimensions: 768
Title: Seoul Tower, Content: Seoul Tower is a communication and observation tower located on Namsan Mountain in central Seoul, South Korea., Embedding Dimensions: 768
Title: Gwanghwamun Gate, Content: Gwanghwamun is t

## Retrieval and Generation

### Define the query and its embedding

In [7]:
query = "Tell me about gates in South Korea."

In [8]:
conn = connect_db()
cur = conn.cursor()
    
# Embed the query using the ollama_embed function
cur.execute("""
    SELECT ollama_embed('nomic-embed-text', %s, _host=>'http://172.17.0.2:11434');
""", (query,))
query_embedding = cur.fetchone()[0]

# Retrieve relevant documents based on cosine distance
cur.execute("""
    SELECT title, content, 1 - (embedding <=> %s) AS similarity
    FROM documents
    ORDER BY similarity DESC
    LIMIT 3;
""", (query_embedding,))

rows = cur.fetchall()
    
# Prepare the context for generating the response
context = "\n\n".join([f"Title: {row[0]}\nContent: {row[1]}" for row in rows])
print(context)

cur.close()
conn.close()

Title: Gwanghwamun Gate
Content: Gwanghwamun is the main and largest gate of Gyeongbokgung Palace, in Jongno-gu, Seoul, South Korea.

Title: Gwanghwamun Gate
Content: Gwanghwamun is the main and largest gate of Gyeongbokgung Palace, in Jongno-gu, Seoul, South Korea.

Title: Dongdaemun Design Plaza
Content: The Dongdaemun Design Plaza is a major urban development landmark in Seoul, South Korea.


In [9]:
conn = connect_db()
cur = conn.cursor()

# Generate the response using the ollama_generate function
cur.execute("""
    SELECT ollama_generate('llama3.2', %s, _host=>'http://172.17.0.2:11434');
""", (f"Query: {query}\nContext: {context}",))
    
model_response = cur.fetchone()[0]
print(model_response['response'])
    
cur.close()
conn.close()

Gates in South Korea play an essential role in the country's rich history and cultural heritage. Here are some interesting facts about gates in South Korea:

1. Historical significance: Gates have been an integral part of Korean palace architecture for centuries, serving as entrance points to royal palaces, fortresses, and other important buildings.
2. Gwanghwamun Gate: The Gwanghwamun Gate is one of the most iconic gates in South Korea, located at Gyeongbokgung Palace in Seoul. It is considered a symbol of Korean culture and heritage, representing the country's rich history and traditions.

3. Architectural style: Traditional Korean gates were often designed with a distinctive shape, featuring a curved or arched roof, wooden posts, and decorative carvings. The Gwanghwamun Gate is a prime example of this architectural style.

4. Modern gates: In recent years, modern gates have become an integral part of South Korea's urban landscape. The Dongdaemun Design Plaza in Seoul is a notable ex